shape of image:

```
SCREEN_WIDTH = 450
SCREEN_HEIGHT = 200
```


In [33]:
SCREEN_WIDTH = 450
SCREEN_HEIGHT = 200

CLASStorch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

input: (N, C_in, H_in, W_in)

output: (N, C_out, H_out, W_out)

$$H_{out} = \frac{H_{in} + 2 \times padding[0] - dilation[0] \times (kernel\_size[0] -1) -1}{stride[0]}+1 $$

In [34]:
import torch
from torch import nn

class CNN(nn.Module):
    def __init__(self, out_channels=6, kernel_size=1, stride=1, padding=0, normalization=False):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=out_channels, 
            kernel_size=kernel_size, stride=stride, padding=padding
        )
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(135000, 5) 

    def forward(self, x):
        print("original shape:", x.shape)

        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        print("after C shape:", x.shape)
        x = x.view(x.size(0), -1)
        print("after view:", x.shape)
        x = self.fc1(x)
        return x


In [35]:
import torch.optim as optim
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Read Image

In [36]:
from torchvision import transforms
transform_norm = transforms.Compose([
    transforms.ToTensor(),
])

image_path = "../converttext/imgdataset/5AMPLETyponoised.jpg"
img = Image.open(image_path)
dog = transform_norm(img)
# dog = normalize(dog)
dog = torch.unsqueeze(dog, 0)
output = model(dog.to(device))
print(output)

original shape: torch.Size([1, 3, 200, 450])
after C shape: torch.Size([1, 6, 100, 225])
after view: torch.Size([1, 135000])
tensor([[ 0.3846, -0.1179, -0.1451, -0.5415,  0.4295]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


Tokenizer

In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
paths = ["../babylm_data/babylm_100M/all_100_zzyyh.train"]
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

Train

dataset / dataloader

pipeline